# Simple Graph
Ok, it's time to dive into LangGraph now that you have some of the fundamental concepts down.  We will build a simple graph with 3 modes and one conditional edge.

# What are nodes
Nodes can be tools, agents or another graph (sub-graph), technically they are really just python functions.  When the team Agent is used, it often just means that the Agent will interact with an LLM, but there no requirement that an Agent **must** make an LLM call.  Graphs can be using without any interaction with an LLM

# State
The State Schema servies as the input schema for all Nodes and Edges in the graph.  Basically, Nodes will read and write to the State and this is how the various node understand what the previous Nodes have done or need to have done.  So, lets define the State object now.

In [1]:
from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str

# Nodes
The first positional argument si the state, as defined above.  Because the state is a **TypedDict** with the schema as defined above, each node can acces th key, **graph_state, with **state['graph_state']**.

Each node returns a new value of the state key **graph_state**.

By default, the new value returned by each node will override the prior state value.

In [2]:
def node_1(state):
    print("---Node 1---")
    return {"graph_state": state['graph_state'] +" I am"}

def node_2(state):
    print("---Node 2---")
    return {"graph_state": state['graph_state'] +" happy!"}

def node_3(state):
    print("---Node 3---")
    return {"graph_state": state['graph_state'] +" sad!"}

# Edges
**Edges** connect the nodes.  Normal **Edges** are used if you want to *always* go from, for example, **node_1** to **node_2**.

**Conditional Edges** are used when you need to *optionally* route between nodes, keyword here is *optionally*.  **Conditional Edges** are implemented as functions that return the next node to vist based upon some logic.

In [3]:
import random
from typing import Literal

def decide_mood(state) -> Literal["node_2", "node_3"]:
    
    # Often, we will use state to decide on the next node to visit
    user_input = state['graph_state'] 
    
    # Here, let's just do a 50 / 50 split between nodes 2, 3
    if random.random() < 0.5:

        # 50% of the time, we return Node 2
        return "node_2"
    
    # 50% of the time, we return Node 3
    return "node_3"

# Graph Construction
Now, we will build the graph from our components defined above.  The **State** class is the class that we can use to initialzie the **StateGraph**.

1. We need to initialize a StateGraph with the *State* calls we defined above.
2. Then we add our nodes and edges.
3. We will use the **START** Node, a special node that sends user input to the graph, to indicate where to start our graph.
4. The *END* Node is a special node that represenbts a terminal node.
5. Finally, we compile our graph to perform a few basic checks on the graph structure.
6. We can visualiuze the graph as a Mermaid diagram.

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

# Graph Invocation
The compiled graphe implements the [runnable](https://python.langchain.com/v0.1/docs/expression_language/interface/) protocol which provides a standard way to execute LangChain components, **invoke** is one of the standard methods in this interface.

The **input** is a dictionary **{"graph_state":"Hi, this is Rick."}**, which sets the initial value for our graph state dictionary.

When **invoke** is called, the graph starts execution from the **START** node.  It progresses through the defined nodes (**node_1**, **node_2**, **node_3**) in order.

The **conditional edge** will traverse from **node_1** to **node_2** or **node_3** using a 50/50 decison rule.

Each node funciton receives the current state and returns the new value, which overrides the graph state.  The execution continues until it reaches the **END** node.

In [17]:
graph.invoke({"graph_state" : "Hi, this is Rick."})

---Node 1---
---Node 3---


{'graph_state': 'Hi, this is Rick. I am sad!'}

**invoke** runs the entire graph synchronously!  This waits for each step to complete before moving to the next.  It returns the final state of the graph after all the nodes have executed:

   ~~~
      {'graph_state': 'Hi, this is Rick. I am sad!'}
   ~~~